In [122]:
import guitarpro
import music21
import math
from os import listdir


def get_note(track, note):
    base_note = track.strings[note.string - 1]
    n1 = music21.note.Note(str(base_note))
    return n1.transpose(note.value)

# unique note number
def diatonicNoteName(note): 
    p1 = music21.pitch.Pitch(note.nameWithOctave)
    return p1.diatonicNoteNum
    
# midi note number
def midi(note): 
    p1 = music21.pitch.Pitch(note.nameWithOctave)
    return p1.midi
    
# midi note number
def frequency(note, multipler = 100): 
    p1 = music21.pitch.Pitch(note.nameWithOctave)
    return round(p1.frequency * multipler)      

def get_duration(beat):
    value = 256 / beat.duration.value
    if beat.duration.isDotted:
        value = value * 1.5
    return int(value)

#https://www.midi.org/specifications/item/gm-level-1-sound-set - id for instrument
def is_bass_midi_instrument(instrument):
    if instrument in range(33, 40):
        return True
    return False

def get_bass_track(song):
    for track in song.tracks:
        if is_bass_midi_instrument(track.channel.instrument) or "bass" in track.name.lower():
            return track        
    
row = {
    "artist": None,
    "album" : None,
    "title" : None,
    "key"   : None,
    "tempo" : None,
    "note"  : None,
    "frequency"  : None,
    "duration" : None,
    "hammer" : None,
    "ghostNote" : None,
    "slide" : None
} 


output = open('output/output.csv', 'w')    
#write header
seperator = ', '
#print(seperator.join(row))
output.write(seperator.join(row) + '\n')

#parsing files
datadir = 'data/'
files = listdir(datadir)
for gpfile in files:   
    song = guitarpro.parse(datadir + gpfile)
    track = get_bass_track(song)

    row["artist"] = song.artist
    row["album"]  = song.album
    row["title"]  = song.title
    row["key"]    = song.key.name
    row["tempo"]  = str(song.tempo)

    for measure in track.measures:
        for voice in measure.voices:
            # measure
            for beat in voice.beats:
                row["duration"] = str(get_duration(beat))
                if beat.status == guitarpro.BeatStatus.rest:
                    row["note"] = "R"
                    row["frequency"] = str(0)
                else:
                    for note in beat.notes:                       
                        musicnote = get_note(track, note)
                        row["note"] = musicnote.nameWithOctave
                        row["frequency"] = str(frequency(musicnote))
                        row["hammer"] = str(effect.hammer)
                        row["ghostNote"] = str(effect.ghostNote)
                        row["slide"] = str(len(note.effect.slides))
                        
                #print(row.values())
                output.write(seperator.join(row.values()) + '\n')
#close file                
output.close()